Следующие 2 ячейки выполнять не нужно. Они нужны лишь для работы в Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/My Drive/Colab Notebooks/data

/content/drive/My Drive/Colab Notebooks/data


Запускать можно код, начиная со следующей ячейки

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
flags_descripts = ["country", "zone", "square", "population", "continent_by_square", "continent_by_population","language","religion",
                   "vertical_count","horizontal_count","colors_count","has_red","has_green","has_blue","has_gold","has_white",
                   "has_black","has_orange", "dominant_color"]

In [4]:
df_train = pd.read_csv('Flags.csv', names=flags_descripts, sep=";")
df_train.shape

(194, 19)

In [5]:
df_train

,country,zone,square,population,continent_by_square,continent_by_population,language,religion,vertical_count,horizontal_count,colors_count,has_red,has_green,has_blue,has_gold,has_white,has_black,has_orange,dominant_color
0,Afghanistan,1,648,16,1,1,10,2,0,3,5,1,1,0,1,1,1,0,green
1,Albania,1,29,3,5,3,6,6,0,0,3,1,0,0,1,0,1,0,red
2,Algeria,1,2388,20,2,2,8,2,2,0,3,1,1,0,0,1,0,0,green
3,American-Samoa,3,0,0,6,6,1,1,0,0,5,1,0,1,1,1,0,1,blue
4,Andorra,1,0,0,5,3,6,0,3,0,3,1,0,1,1,0,0,0,gold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Western-Samoa,3,3,0,6,6,1,1,0,0,3,1,0,1,0,1,0,0,red
190,Yugoslavia,1,256,22,5,3,6,6,0,3,4,1,0,1,1,1,0,0,red
191,Zaire,2,905,28,2,2,10,5,0,0,4,1,1,0,1,0,0,1,green
192,Zambia,2,753,6,2,2,10,5,3,0,4,1,1,0,0,0,1,1,green


In [6]:
name_scale = ["country", "zone", "language", "has_red", "has_green", "has_blue", "has_gold", "has_white", "has_black", "has_orange", "dominant_color"]
strong_scale = ["square", "population", "vertical_count", "horizontal_count","colors_count"]
order_scale = ["continent_by_square", "continent_by_population"]


Три функции для вычисления расстояния между объектами с учётом типов шкал

In [7]:
def calculate_strong_scale_dists(np_strong_scale_array, dists):
  
  np_strong_scale_array = np_strong_scale_array[:,np.newaxis]
  
  for i in range(np_strong_scale_array.shape[0]):
     dists[i] = np.sum(np.abs(np_strong_scale_array - np_strong_scale_array[i]) / np.abs(np.max(np_strong_scale_array) - np.min(np_strong_scale_array)), axis = 1)  
  

In [8]:
def calculate_name_scale_dists(np_name_scale_array, dists):
  
  for i in range(np_name_scale_array.shape[0]):
    for j in range(np_name_scale_array.shape[0]):
      for k in range(np_name_scale_array.shape[0]):
        if (((np_name_scale_array[i] == np_name_scale_array[k]) and (np_name_scale_array[j] != np_name_scale_array[k]))
        or ((np_name_scale_array[i] != np_name_scale_array[k]) and (np_name_scale_array[j] == np_name_scale_array[k]))):
          dists[i][j] += 1.0
  dists /= np_name_scale_array.shape[0]

In [9]:
def calculate_order_scale_dists(np_order_scale_array, dists):
  for i in range(np_order_scale_array.shape[0]):
    for j in range(np_order_scale_array.shape[0]):
      for k in range(np_order_scale_array.shape[0]):

        if (((np_order_scale_array[i] > np_order_scale_array[k]) and (np_order_scale_array[j] < np_order_scale_array[k]))
        or ((np_order_scale_array[i] < np_order_scale_array[k]) and (np_order_scale_array[j] > np_order_scale_array[k]))):
          dists[i][j] += 1.0

        elif (((np_order_scale_array[i] != np_order_scale_array[k]) and (np_order_scale_array[j] == np_order_scale_array[k]))
        or ((np_order_scale_array[i] == np_order_scale_array[k]) and (np_order_scale_array[j] != np_order_scale_array[k]))):
          dists[i][j] += 0.5
  dists /= (np_order_scale_array.shape[0] - 1)

Вызываем данные функции и получаем кучу матриц расстояний, которые необходимо просуммировать для получения попарного расстояния между объектами

In [10]:
np_name_scale_dists = np.zeros((len(name_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in name_scale:
  calculate_name_scale_dists(np.array(df_train[i]), np_name_scale_dists[name_scale.index(i)])

np_order_scale_dists = np.zeros((len(order_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in order_scale:
   calculate_order_scale_dists(np.array(df_train[i]), np_order_scale_dists[order_scale.index(i)])

np_strong_scale_dists = np.zeros((len(strong_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in strong_scale:
   calculate_strong_scale_dists(np.array(df_train[i]), np_strong_scale_dists[strong_scale.index(i)])


Проверяем основные свойства метрик

In [15]:
for j in range(df_train.shape[0]): 
  for k in range(df_train.shape[0]):
    for i in range(len(strong_scale)):
      if (j == k):
        assert np_strong_scale_dists[i][j][k] == 0.0
      else:
        assert np_strong_scale_dists[i][j][k] == np_strong_scale_dists[i][k][j]
    for i in range(len(name_scale)):
      if (j == k):
        assert np_name_scale_dists[i][j][k] == 0.0
      else:
        assert np_name_scale_dists[i][j][k] == np_name_scale_dists[i][k][j]
    for i in range(len(order_scale)):
      if (j == k):
        assert np_order_scale_dists[i][j][k] == 0.0
      else:
        assert np_order_scale_dists[i][j][k] == np_order_scale_dists[i][k][j]


Вычисление попарного расстояния между объектами

In [12]:
dists = np.zeros((df_train.shape[0], df_train.shape[0]), np.float64)

for i in range(len(name_scale)):
  dists += np.power(np_name_scale_dists[i], 2)

for i in range(len(order_scale)):
  dists += np.power(np_order_scale_dists[i], 2)

for i in range(len(strong_scale)):
  dists += np.power(np_strong_scale_dists[i], 2)

n = len(name_scale) + len(order_scale) + len(strong_scale)
dists = math.sqrt(1/n) * np.power(dists,0.5)
print (dists)

[[0.         0.42655102 0.37395208 ... 0.44475186 0.46668589 0.17883936]
 [0.42655102 0.         0.51571292 ... 0.47788556 0.49831116 0.43589684]
 [0.37395208 0.51571292 0.         ... 0.45214499 0.44477872 0.40788746]
 ...
 [0.44475186 0.47788556 0.45214499 ... 0.         0.36214091 0.42631721]
 [0.46668589 0.49831116 0.44477872 ... 0.36214091 0.         0.44912133]
 [0.17883936 0.43589684 0.40788746 ... 0.42631721 0.44912133 0.        ]]


Находим объект, суммарное расстояние до которого от всех остальных объектов максимально.

In [13]:
np_summary_dists = np.zeros(dists.shape[0], np.float64)
np_summary_dists = np.sum(dists, axis = 1)
max = np.max(np_summary_dists)
print(max)
print(np_summary_dists.tolist().index(max))

105.40089193430285
157
